# Semantic Caching

for example if a customer is asking "what is my bank balance?", this is not a query that you would want cached. Similarly, 

This example requires a Redis Cache to be set up. This can be done using Azure Cache for Redis, with instructions in the link below. Alternatively, you can simply read through the notebook and observe the speed up that was achieved.

https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/cache-tutorial-semantic-cache

LangChain provides a simple interface for managing caching:

https://python.langchain.com/docs/integrations/llms/llm_caching

# Use Case: Answering common questions

In [1]:
import openai
import redis
import os
import langchain
from langchain.llms import AzureOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.globals import set_llm_cache
from langchain.cache import RedisSemanticCache
import time
from dotenv import load_dotenv
import os
from langchain.globals import set_llm_cache
from langchain_openai import AzureChatOpenAI


# Load environment variables
load_dotenv()

AZURE_ENDPOINT=os.getenv("AZURE_ENDPOINT")
API_KEY=os.getenv("API_KEY")
API_VERSION=os.getenv("API_VERSION")
LLM_DEPLOYMENT_NAME=os.getenv("MODELGPT432k")
LLM_MODEL_NAME=os.getenv("MODELGPT432k")
EMBEDDINGS_DEPLOYMENT_NAME=os.getenv("EMBEDDING")
EMBEDDINGS_MODEL_NAME=os.getenv("EMBEDDING")
REDIS_ENDPOINT=os.getenv("REDIS_ENDPOINT")
REDIS_PASSWORD=os.getenv("REDIS_PASSWORD")
os.environ["OPENAI_API_VERSION"] = API_VERSION
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = API_KEY

In [2]:
llm = AzureChatOpenAI(
    deployment_name=LLM_MODEL_NAME,
)
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    model=EMBEDDINGS_MODEL_NAME,
)

In [4]:
redis_url = "rediss://:" + REDIS_PASSWORD + "@"+ REDIS_ENDPOINT
set_llm_cache(RedisSemanticCache(redis_url = redis_url, embedding=embeddings, score_threshold=0.01))

### A: Base Case

**Time taken: 18 seconds**

A standard LLM call is made.

In [ ]:
llm.invoke("How can I sign up for Microsoft Teams?")

AIMessage(content='You can sign up for Azure by following these steps:\n\n1. Go to the Azure website (azure.microsoft.com)\n2. Click on the "Start free" button on the home page.\n3. You will be redirected to the sign-up page, where you\'ll need to provide your Microsoft account credentials. If you do not have a Microsoft account, you can create one by clicking on "Create one!".\n4. After you log in or create a new account, you\'ll need to verify your identity through a phone number and provide some information about yourself.\n5. You will also need a valid credit card for verification. Microsoft states that you will not be charged unless you decide to upgrade.\n6. After you\'ve filled in all of the required information, click on the "Sign Up" button.\n7. You have now signed up for Azure. You can go to the dashboard and start using the services. \n\nRemember, you get $200 credit for the first 30 days and a limited number of free services for a year.', response_metadata={'token_usage': {

### B: Leveraging Semantic Caching

**Time taken: 18 seconds**

The first time the question is asked, it takes the standard amount of time. However, subsequent calls that have a sufficient semantic similarity will leverage the cached response, saving significant amounts of time.

#### First request (not cached)

With GPT 4 32k, this may take ~18 seconds, as it is actually calling Azure OpenAI.

In [6]:
llm.invoke("How can I sign up for Azure?")

AIMessage(content='You can sign up for Azure by following these steps:\n\n1. Go to the Azure website (azure.microsoft.com)\n2. Click on the "Start free" button on the home page.\n3. You will be redirected to the sign-up page, where you\'ll need to provide your Microsoft account credentials. If you do not have a Microsoft account, you can create one by clicking on "Create one!".\n4. After you log in or create a new account, you\'ll need to verify your identity through a phone number and provide some information about yourself.\n5. You will also need a valid credit card for verification. Microsoft states that you will not be charged unless you decide to upgrade.\n6. After you\'ve filled in all of the required information, click on the "Sign Up" button.\n7. You have now signed up for Azure. You can go to the dashboard and start using the services. \n\nRemember, you get $200 credit for the first 30 days and a limited number of free services for a year.', response_metadata={'token_usage': {

#### Second request, exact match (cached)

As the second request is cached, it returns in only ~1-2 seconds. This was the same question, where every character was an exact match.

In [7]:
llm.invoke("How can I sign up for Azure?")

AIMessage(content='You can sign up for Azure by following these steps:\n\n1. Go to the Azure website (azure.microsoft.com)\n2. Click on the "Start free" button on the home page.\n3. You will be redirected to the sign-up page, where you\'ll need to provide your Microsoft account credentials. If you do not have a Microsoft account, you can create one by clicking on "Create one!".\n4. After you log in or create a new account, you\'ll need to verify your identity through a phone number and provide some information about yourself.\n5. You will also need a valid credit card for verification. Microsoft states that you will not be charged unless you decide to upgrade.\n6. After you\'ve filled in all of the required information, click on the "Sign Up" button.\n7. You have now signed up for Azure. You can go to the dashboard and start using the services. \n\nRemember, you get $200 credit for the first 30 days and a limited number of free services for a year.', id='run-43daa71f-9d5d-4095-8de2-1f8

#### Third request, semantically similar match (cached)

Cached responses do not have to be the exact same input string- semantically similar questions will also match, saving cost and speed.

In [9]:
llm.invoke("I want to sign up for Azure")

AIMessage(content='You can sign up for Azure by following these steps:\n\n1. Go to the Azure website (azure.microsoft.com)\n2. Click on the "Start free" button on the home page.\n3. You will be redirected to the sign-up page, where you\'ll need to provide your Microsoft account credentials. If you do not have a Microsoft account, you can create one by clicking on "Create one!".\n4. After you log in or create a new account, you\'ll need to verify your identity through a phone number and provide some information about yourself.\n5. You will also need a valid credit card for verification. Microsoft states that you will not be charged unless you decide to upgrade.\n6. After you\'ve filled in all of the required information, click on the "Sign Up" button.\n7. You have now signed up for Azure. You can go to the dashboard and start using the services. \n\nRemember, you get $200 credit for the first 30 days and a limited number of free services for a year.', id='run-43daa71f-9d5d-4095-8de2-1f8